In [1]:
import numpy as np # array function purpose
import os #to access datasets
import matplotlib.pyplot as plt #to plot graph
from imutils import paths #to access datasets

from tensorflow.keras.applications import MobileNetV2
#tensorflow->keras->python->application->MobileNetV2

from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input #pip install Keras-Applications

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer #machine learning library
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
datasets=r'C:\Users\Rajdeep Sharma\Desktop\Face Mask Detector\Mask Dataset'
imagePaths=list(paths.list_images(datasets))

In [3]:
imagePaths

[]

In [4]:
data=[]  #x is image
labels=[]  #y is labels
for i in imagePaths:
    label=i.split(os.path.sep)[-1]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)

In [5]:
labels

[]

In [6]:
image.shape

NameError: name 'image' is not defined

In [7]:
data=np.array(data,dtype='float32') # for converting into array format
labels=np.array(labels)

In [8]:
data.shape

(0,)

In [14]:
lb=LabelBinarizer()# for calling a class
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [15]:
data

array([[[[-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         ...,
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.84313726, -0.8980392 , -0.77254903]],

        [[-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         ...,
         [-0.7490196 , -0.8039216 , -0.6784314 ],
         [-0.7490196 , -0.8039216 , -0.6784314 ],
         [-0.7411765 , -0.79607844, -0.67058825]],

        [[-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9529412 , -0.81960785],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         ...,
         [-0.88235295, -0.8980392 , -0.7882353 ],
         [-0.85882354, -0.8901961 , -0.77254903],
         [-0.85882354, -0.8745098 , -0.7647059 ]],

        ...,

        [[ 0.38823533, -0.9607843 , -0

In [16]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [17]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=10)

In [18]:
train_X

array([[[[-0.19215685, -0.3333333 , -0.44313723],
         [-0.19215685, -0.3333333 , -0.44313723],
         [-0.19215685, -0.3333333 , -0.44313723],
         ...,
         [ 0.5058824 ,  0.37254906,  0.2313726 ],
         [ 0.5137255 ,  0.3803922 ,  0.23921573],
         [ 0.5137255 ,  0.3803922 ,  0.23921573]],

        [[-0.20784312, -0.3490196 , -0.4588235 ],
         [-0.20784312, -0.3490196 , -0.4588235 ],
         [-0.20784312, -0.3490196 , -0.4588235 ],
         ...,
         [ 0.5058824 ,  0.37254906,  0.2313726 ],
         [ 0.5137255 ,  0.3803922 ,  0.23921573],
         [ 0.5137255 ,  0.3803922 ,  0.23921573]],

        [[-0.19215685, -0.3333333 , -0.44313723],
         [-0.19215685, -0.3333333 , -0.44313723],
         [-0.19215685, -0.3333333 , -0.44313723],
         ...,
         [ 0.5137255 ,  0.3803922 ,  0.254902  ],
         [ 0.52156866,  0.38823533,  0.26274514],
         [ 0.52156866,  0.38823533,  0.26274514]],

        ...,

        [[-0.7254902 , -0.7411765 , -0

In [19]:
aug=ImageDataGenerator(rotation_range=20.0,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

In [20]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [21]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [25]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)#making layers narrow link:https://www.google.com/search?q=flatten+in+cnn&sxsrf=AOaemvLAWPhOVVbZffmlkxRpR8TOlpdy0w:1642946676240&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjplZaohcj1AhWJF4gKHeKIAZMQ_AUoAXoECAEQAw&biw=1600&bih=732&dpr=1#imgrc=X8AOXX3jw-DunM
headModel=Dense(128,activation='relu')(headModel)# for making neuron links Link:https://www.google.com/search?q=dense+in+cnn&tbm=isch&ved=2ahUKEwiClvWqhcj1AhUv8TgGHS5EAkEQ2-cCegQIABAA&oq=dense+in+cnn&gs_lcp=CgNpbWcQARgAMgUIABCABDIGCAAQBxAeOgcIIxDvAxAnOgYIABAIEB46CAgAEAcQBRAeOggIABAIEAcQHlDyD1jqNGDBRGgDcAB4AIABrwGIAcMOkgEEMC4xNJgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=eWDtYcL1O6_i4-EProiJiAQ&bih=732&biw=1600#imgrc=q647eVAGnQJc0M 
headmodel=Dropout(0.5)(headModel)# link:https://www.google.com/search?q=dropout+in+cnn&tbm=isch&ved=2ahUKEwiG2Nqehsj1AhVx_jgGHfh5BN4Q2-cCegQIABAA&oq=dropout+in+cnn&gs_lcp=CgNpbWcQAzIFCAAQgAQyBggAEAgQHjIECAAQGDIECAAQGDIECAAQGDIECAAQGFDqAljqAmDwBmgAcAB4AIABlQOIAbwEkgEHMC4xLjQtMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=bGHtYYbyMfH84-EP-POR8A0&bih=732&biw=1600#imgrc=8tnv3OpJljdgeM
# dropout is done for avoiding memorization of features like for training set it will perform well bcoz it memorizes instead of learning 
headModel=Dense(2,activation='softmax')(headModel)# this is for cutoff before classifying 
model=Model(inputs=baseModel.input,outputs=headModel)

In [26]:
for layers in baseModel.layers:
    layers.trainable=False

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [31]:
learning_rate=0.0001 #time allotted for learning from a image
Epochs=20  # number of times the model go through a single image
BS=10 # Batch size means number of images held to train the model at a single time
opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

model.fit(
    aug.flow(train_X,train_Y,batch_size=BS),
    steps_per_epoch=len(train_X)//BS,
    validation_data=(train_X,train_Y),
    validation_steps=len(train_X)//BS,
    epochs=Epochs
)

model.save(r'C:\Users\Rajdeep Sharma\Desktop\Face Mask Detector\Mask Dataset\MobileNet_V2.model')

Epoch 1/20
110/110 [==============================] - 442s 3s/step - loss: 0.3390 - accuracy: 0.9118 - val_loss: 0.0995 - val_accuracy: 0.9918
Epoch 2/20
110/110 [==============================] - 325s 3s/step - loss: 0.1113 - accuracy: 0.9855 - val_loss: 0.0451 - val_accuracy: 0.9945
Epoch 3/20
110/110 [==============================] - 345s 3s/step - loss: 0.0697 - accuracy: 0.9891 - val_loss: 0.0310 - val_accuracy: 0.9936
Epoch 4/20
110/110 [==============================] - 323s 3s/step - loss: 0.0503 - accuracy: 0.9873 - val_loss: 0.0244 - val_accuracy: 0.9936
Epoch 5/20
110/110 [==============================] - 207s 2s/step - loss: 0.0436 - accuracy: 0.9873 - val_loss: 0.0191 - val_accuracy: 0.9945
Epoch 6/20
110/110 [==============================] - 206s 2s/step - loss: 0.0404 - accuracy: 0.9891 - val_loss: 0.0167 - val_accuracy: 0.9973
Epoch 7/20
110/110 [==============================] - 207s 2s/step - loss: 0.0272 - accuracy: 0.9927 - val_loss: 0.0137 - val_accuracy: 0.9973

C:\Users\abc\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\abc\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [33]:
predict=model.predict(test_X,batch_size=BS) # predict on test_X 
predict=np.argmax(predict,axis=1) # argmax means which output having maximum probability with mask or without mask
print(classification_report(test_Y.argmax(axis=1),predict,target_names=lb.classes_))# comparing the output of two result
#result from test_y is compared with predict which has the result of test_X
# targets names are 1 0 and 0 1

              precision    recall  f1-score   support

   with_mask       1.00      0.99      0.99       138
without_mask       0.99      1.00      0.99       138

    accuracy                           0.99       276
   macro avg       0.99      0.99      0.99       276
weighted avg       0.99      0.99      0.99       276

